### Building out the Recommender System using the Businesses DataFrame

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import ast

In [2]:
# Load business data
df = pd.read_csv('../Data/business.csv')

In [3]:
df.head()

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,attributes,...,cat_Nicaraguan,cat_Georgian,cat_Czech/Slovakian,cat_Sardinian,cat_PubFood,cat_FoodTrucks,cat_WineTours,cat_FoodTours,cat_Bistros,cat_Drive-ThruBars
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,Boulder,CO,80302,40.017544,-105.283348,4.0,86,"{'RestaurantsTableService': 'True', 'WiFi': ""u...",...,0,0,0,0,0,0,0,0,0,0
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,Portland,OR,97218,45.588906,-122.593331,4.0,126,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...",...,0,0,0,0,0,0,0,0,0,0
2,D4JtQNTI4X3KcbzacDJsMw,Bob Likes Thai Food,Vancouver,BC,V5V,49.251342,-123.101333,3.5,169,"{'GoodForKids': 'True', 'Alcohol': ""u'none'"", ...",...,0,0,0,0,0,0,0,0,0,0
3,jFYIsSb7r1QeESVUnXPHBw,Boxwood Biscuit,Columbus,OH,43206,39.947007,-82.997471,4.5,11,NaN,...,0,0,0,0,0,0,0,0,0,0
4,rYs_1pNB_RMtn5WQh55QDA,Chautauqua General Store,Boulder,CO,80302,39.998449,-105.281006,3.5,5,"{'BikeParking': 'True', 'RestaurantsTakeOut': ...",...,0,0,0,0,0,0,0,0,0,0


### Prepping the Data

In [4]:
small_df = df[['business_id', 'name', 'city', 'state', 'postal_code', 'attributes']].copy()

In [5]:
small_df.head()

,business_id,name,city,state,postal_code,attributes
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,Boulder,CO,80302,"{'RestaurantsTableService': 'True', 'WiFi': ""u..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,Portland,OR,97218,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt..."
2,D4JtQNTI4X3KcbzacDJsMw,Bob Likes Thai Food,Vancouver,BC,V5V,"{'GoodForKids': 'True', 'Alcohol': ""u'none'"", ..."
3,jFYIsSb7r1QeESVUnXPHBw,Boxwood Biscuit,Columbus,OH,43206,NaN
4,rYs_1pNB_RMtn5WQh55QDA,Chautauqua General Store,Boulder,CO,80302,"{'BikeParking': 'True', 'RestaurantsTakeOut': ..."


In [6]:
# removes the states located in Canada (ABE, and BC)
small_df = small_df[small_df['state'] != 'ABE']
small_df = small_df[small_df['state'] != 'BC']

In [7]:
# drops null attributes
small_df = small_df[small_df['attributes'].notnull()]

In [8]:
# drops null postal_codes
small_df = small_df[small_df['postal_code'].notnull()]

In [9]:
# Checks the state distribution
small_df['state'].value_counts()

MA    8710
FL    7013
OR    5767
GA    4998
TX    4379
OH    3974
WA     739
CO     682
NH       2
DE       1
KS       1
VA       1
KY       1
MN       1
WY       1
Name: state, dtype: int64

In [10]:
# Subsets the data to only include Ohio restaurant businesses
small_df = small_df[small_df['state'] == 'OH']

In [11]:
# Checks the shape to make sure it subsetted correctly
small_df.shape

(3974, 6)

In [12]:
# Checks the attributes column data type
small_df['attributes'].dtype

dtype('O')

In [13]:
# Turns the attributes column into actual dictionaries
small_df['attributes'] = small_df['attributes'].map(lambda x: ast.literal_eval(x))

In [14]:
# Resets the index of small_df
small_df.reset_index(drop=True, inplace=True)

In [15]:
small_df.head()

,business_id,name,city,state,postal_code,attributes
0,5eq56X-e9YbAKmHgUOyXSg,Hunan Lion,Columbus,OH,43220,"{'GoodForKids': 'True', 'RestaurantsGoodForGro..."
1,Fl_7YDOMt58rTNHa7p2G4w,Waffle House,Urbancrest,OH,43123,"{'OutdoorSeating': 'False', 'RestaurantsAttire..."
2,vecuat0jOia-CJveW3ngDw,Schmidt's Sausage Haus,Columbus,OH,43206,"{'GoodForKids': 'True', 'HasTV': 'False', 'Res..."
3,ZaMMUcOIngBCgxuKxFzfqg,McDonald's,Columbus,OH,43228,"{'RestaurantsTakeOut': 'True', 'NoiseLevel': '..."
4,C1Jkjsqoz1AsYJy28IosQg,La Michoacana Market,Columbus,OH,43232,"{'Caters': 'True', 'BusinessAcceptsCreditCards..."


In [16]:
small_df.shape

(3974, 6)

In [17]:
# Checks to  make sure that the length of unique business id matches the number of rows in data frame
len(np.unique(small_df['business_id']))

3974

### Cleaning the `attributes` Column

In [18]:
# creates a list of the dicts from the attributes column
attributes_list_of_dicts = list(small_df['attributes'])

In [19]:
# Creates an attributes dataframe where each attribute becomes a column
attribute_df = pd.DataFrame(attributes_list_of_dicts)

In [20]:
attribute_df.head()

,GoodForKids,RestaurantsGoodForGroups,BikeParking,RestaurantsTakeOut,OutdoorSeating,RestaurantsReservations,RestaurantsPriceRange2,Caters,BusinessAcceptsCreditCards,WiFi,...,GoodForDancing,Music,Smoking,Corkage,BYOB,CoatCheck,BestNights,RestaurantsCounterService,AgesAllowed,DietaryRestrictions
0,True,True,True,True,False,True,2,False,True,u'no',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,NaN,True,False,False,1,False,True,u'no',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,True,True,None,False,2,True,True,u'no',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,True,NaN,True,False,False,1,False,True,u'free',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,True,True,NaN,NaN,NaN,True,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Looks at all the columns
attribute_df.columns

Index(['GoodForKids', 'RestaurantsGoodForGroups', 'BikeParking',
       'RestaurantsTakeOut', 'OutdoorSeating', 'RestaurantsReservations',
       'RestaurantsPriceRange2', 'Caters', 'BusinessAcceptsCreditCards',
       'WiFi', 'RestaurantsDelivery', 'HasTV', 'BusinessParking', 'Alcohol',
       'RestaurantsTableService', 'RestaurantsAttire', 'Ambience',
       'NoiseLevel', 'GoodForMeal', 'BYOBCorkage', 'ByAppointmentOnly',
       'WheelchairAccessible', 'HappyHour', 'DogsAllowed',
       'BusinessAcceptsBitcoin', 'DriveThru', 'AcceptsInsurance',
       'GoodForDancing', 'Music', 'Smoking', 'Corkage', 'BYOB', 'CoatCheck',
       'BestNights', 'RestaurantsCounterService', 'AgesAllowed',
       'DietaryRestrictions'],
      dtype='object')

In [22]:
# Drops columns that aren't needed
attribute_df.drop(columns=['BikeParking', 'BusinessParking', 'DogsAllowed', 'ByAppointmentOnly', 'BestNights', 'CoatCheck', 'Smoking', 'Corkage', 'BYOB', 'BYOBCorkage',
                           'AcceptsInsurance'], inplace=True)

In [23]:
attribute_df.head()

,GoodForKids,RestaurantsGoodForGroups,RestaurantsTakeOut,OutdoorSeating,RestaurantsReservations,RestaurantsPriceRange2,Caters,BusinessAcceptsCreditCards,WiFi,RestaurantsDelivery,...,GoodForMeal,WheelchairAccessible,HappyHour,BusinessAcceptsBitcoin,DriveThru,GoodForDancing,Music,RestaurantsCounterService,AgesAllowed,DietaryRestrictions
0,True,True,True,False,True,2,False,True,u'no',False,...,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,True,False,False,1,False,True,u'no',False,...,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,True,None,False,2,True,True,u'no',True,...,"{'dessert': None, 'latenight': False, 'lunch':...",False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,False,True,True,False,False,1,False,True,u'free',True,...,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,True,NaN,NaN,NaN,True,True,NaN,NaN,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* 0: good_for_lunch
* 1: good_for_dinner
* 2: takes_reservations
* 3: outdoor_seating
* 4: restaurant_is_expensive
* 5: has_alcohol
* 6: has_table_service
* 7: ambience_is_classy
* 8: good_for_kids

Maybe also: 
* 9: good_for_groups 
* 10: attire_casual / attire_dressy / attire_formal

In [24]:
attribute_df.dtypes

GoodForKids                   object
RestaurantsGoodForGroups      object
RestaurantsTakeOut            object
OutdoorSeating                object
RestaurantsReservations       object
RestaurantsPriceRange2        object
Caters                        object
BusinessAcceptsCreditCards    object
WiFi                          object
RestaurantsDelivery           object
HasTV                         object
Alcohol                       object
RestaurantsTableService       object
RestaurantsAttire             object
Ambience                      object
NoiseLevel                    object
GoodForMeal                   object
WheelchairAccessible          object
HappyHour                     object
BusinessAcceptsBitcoin        object
DriveThru                     object
GoodForDancing                object
Music                         object
RestaurantsCounterService     object
AgesAllowed                   object
DietaryRestrictions           object
dtype: object

Categories to use for recommender system: 
* kid_friendly
* good_for_groups
* outdoor_seating
* takes_reservations
* has_alcohol
* has_table_service
* ambience_class   (which will be numbers)   --> model with map predictions to these numbers
* good_for_lunch
* good_for_dinner

In [25]:
# Creates a for loop that value_counts each column
for col in attribute_df.columns: 
    print(col)
    print(attribute_df[col].value_counts(dropna=False))

GoodForKids
True     2259
NaN      1361
False     354
Name: GoodForKids, dtype: int64
RestaurantsGoodForGroups
True     2341
NaN      1275
False     358
Name: RestaurantsGoodForGroups, dtype: int64
RestaurantsTakeOut
True     3265
NaN       400
False     264
None       45
Name: RestaurantsTakeOut, dtype: int64
OutdoorSeating
False    1419
True     1356
NaN      1051
None      148
Name: OutdoorSeating, dtype: int64
RestaurantsReservations
False    2213
NaN      1145
True      603
None       13
Name: RestaurantsReservations, dtype: int64
RestaurantsPriceRange2
1      1837
2      1535
NaN     520
3        77
4         5
Name: RestaurantsPriceRange2, dtype: int64
Caters
True     1409
NaN      1376
False    1188
None        1
Name: Caters, dtype: int64
BusinessAcceptsCreditCards
True     3672
NaN       257
False      44
None        1
Name: BusinessAcceptsCreditCards, dtype: int64
WiFi
NaN        1251
u'free'     966
u'no'       702
'free'      591
'no'        436
u'paid'      12
'paid'     

In [26]:
# Columns to keep for recommender
keep_cols = ['GoodForKids', 'RestaurantsGoodForGroups', 'OutdoorSeating', 'RestaurantsReservations', 'Alcohol', 'RestaurantsTableService', 'Ambience', 'GoodForMeal']

In [27]:
# Keeps only the columns that are going to be used for the recommender
new_attribute_df = attribute_df[keep_cols]

In [28]:
# renames the columns to have 'att' as a prefix
new_attribute_df.columns = [f'att_{col}' for col in new_attribute_df.columns]

In [29]:
new_attribute_df.head()

,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_Alcohol,att_RestaurantsTableService,att_Ambience,att_GoodForMeal
0,True,True,False,True,u'full_bar',True,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'..."
1,True,True,False,False,u'none',NaN,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'..."
2,True,True,None,False,u'full_bar',NaN,"{'touristy': None, 'hipster': False, 'romantic...","{'dessert': None, 'latenight': False, 'lunch':..."
3,False,True,False,False,u'none',False,"{'romantic': False, 'intimate': False, 'classy...","{'dessert': False, 'latenight': False, 'lunch'..."
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
new_attribute_df.shape

(3974, 8)

In [31]:
# Drops rows that are completely all null
new_attribute_df.dropna(how='all', inplace=True)

<ipython-input-31-fdec2ae114a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_attribute_df.dropna(how='all', inplace=True)


In [32]:
# Checks the shape to make sure number of rows matches the number of rows in the original small_df dataframe of businesses
new_attribute_df.shape

(3342, 8)

### Cleaning the Ambience and GoodForMeal column after merging back with small_df

In [33]:
# Merges back small_df with new_attribute_df.  Merges on index
merged_small_df = pd.merge(small_df, new_attribute_df, left_index=True, right_index=True)

In [34]:
small_df.shape, new_attribute_df.shape

((3974, 6), (3342, 8))

In [35]:
merged_small_df.shape

(3342, 14)

In [36]:
merged_small_df.head()

,business_id,name,city,state,postal_code,attributes,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_Alcohol,att_RestaurantsTableService,att_Ambience,att_GoodForMeal
0,5eq56X-e9YbAKmHgUOyXSg,Hunan Lion,Columbus,OH,43220,"{'GoodForKids': 'True', 'RestaurantsGoodForGro...",True,True,False,True,u'full_bar',True,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'..."
1,Fl_7YDOMt58rTNHa7p2G4w,Waffle House,Urbancrest,OH,43123,"{'OutdoorSeating': 'False', 'RestaurantsAttire...",True,True,False,False,u'none',NaN,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'..."
2,vecuat0jOia-CJveW3ngDw,Schmidt's Sausage Haus,Columbus,OH,43206,"{'GoodForKids': 'True', 'HasTV': 'False', 'Res...",True,True,None,False,u'full_bar',NaN,"{'touristy': None, 'hipster': False, 'romantic...","{'dessert': None, 'latenight': False, 'lunch':..."
3,ZaMMUcOIngBCgxuKxFzfqg,McDonald's,Columbus,OH,43228,"{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",False,True,False,False,u'none',False,"{'romantic': False, 'intimate': False, 'classy...","{'dessert': False, 'latenight': False, 'lunch'..."
5,FrFDIPBzRbFVfanxG4wcNQ,Exotic Latin Grill,Columbus,OH,43202,"{'Alcohol': 'u'none'', 'NoiseLevel': ''average...",True,True,True,False,u'none',False,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': True, 'lunch':..."


In [37]:
merged_small_df.reset_index(drop=True, inplace=True)

In [38]:
merged_small_df.isnull().sum()

business_id                        0
name                               0
city                               0
state                              0
postal_code                        0
attributes                         0
att_GoodForKids                  729
att_RestaurantsGoodForGroups     643
att_OutdoorSeating               419
att_RestaurantsReservations      513
att_Alcohol                      717
att_RestaurantsTableService     2125
att_Ambience                     559
att_GoodForMeal                 1207
dtype: int64

In [39]:
merged_small_df[merged_small_df['att_Ambience'].isnull()]

,business_id,name,city,state,postal_code,attributes,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_Alcohol,att_RestaurantsTableService,att_Ambience,att_GoodForMeal
12,ALP7rrHJxXIHyzMMPGQ1vw,LittleMoon Cafe and Tea,Columbus,OH,43202,"{'RestaurantsTakeOut': 'True', 'WheelchairAcce...",NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
14,ukWju178_R0Htx7LBfJFkA,Starbucks,Columbus,OH,43215,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
20,-kcM672owPBvJzDqQ4My_Q,What the Waffle,Columbus,OH,43205,"{'RestaurantsTableService': 'False', 'Restaura...",NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
38,DwkhuFNZXsnEkox39w99Ug,Sloopy's Pub,Hilliard,OH,43026,"{'OutdoorSeating': 'False', 'HappyHour': 'True...",NaN,True,False,NaN,NaN,NaN,NaN,NaN
39,4KZm9EcnygkFmEWpPSNqIw,Hyatt Regency Columbus,Columbus,OH,43215,"{'RestaurantsReservations': 'True', 'Restauran...",NaN,True,NaN,True,NaN,NaN,NaN,"{'dessert': False, 'latenight': False, 'lunch'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,gsVv6sRLTC2FWMpQt38EJA,Royal Burger,Columbus,OH,43228,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
3319,NcNBXf06ueP47KSaA4Jbig,CMYE Meet The Owner: Beer Tasting & Board Games,Columbus,OH,43212,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
3322,sdWYzrG80ZdG010NFEd_gA,Tim Hortons,Columbus,OH,43222,"{'BusinessParking': '{'garage': False, 'street...",True,True,False,False,u'none',NaN,NaN,NaN
3333,WxpzviM3pY8CQABSJy0AgA,Starbucks,Upper Arlington,OH,43221,"{'WiFi': 'u'free'', 'BusinessAcceptsCreditCard...",NaN,NaN,True,NaN,NaN,NaN,NaN,NaN


#### A) Cleans the Ambience column

##### 1) Separates null rows and not null rows in Ambience column

In [40]:
# Splits out the rows that are null and notnull in the att_Ambience column
notnull_ambi_df = merged_small_df[merged_small_df['att_Ambience'].notnull()]
null_ambi_df = merged_small_df[merged_small_df['att_Ambience'].isnull()]

In [41]:
# Turns each value in att_Ambience column into a literal dictionary
notnull_ambi_df['att_Ambience'] = notnull_ambi_df.loc[:,'att_Ambience'].map(lambda x: ast.literal_eval(x))

<ipython-input-41-86f8f22ac94b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notnull_ambi_df['att_Ambience'] = notnull_ambi_df.loc[:,'att_Ambience'].map(lambda x: ast.literal_eval(x))


In [42]:
null_ambi_df.shape

(559, 14)

In [43]:
notnull_ambi_df.shape

(2783, 14)

In [44]:
# Checks to make sure the data type is a dictionary type
type(notnull_ambi_df['att_Ambience'][0])

dict

In [45]:
# Concatenates the notnull and null att_Ambience dataframes back together
merged_small_df = pd.concat([notnull_ambi_df, null_ambi_df])

In [46]:
# Sorts the dataframe by index
merged_small_df.sort_index(inplace=True)

In [47]:
merged_small_df.shape

(3342, 14)

In [48]:
merged_small_df.head()

,business_id,name,city,state,postal_code,attributes,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_Alcohol,att_RestaurantsTableService,att_Ambience,att_GoodForMeal
0,5eq56X-e9YbAKmHgUOyXSg,Hunan Lion,Columbus,OH,43220,"{'GoodForKids': 'True', 'RestaurantsGoodForGro...",True,True,False,True,u'full_bar',True,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'..."
1,Fl_7YDOMt58rTNHa7p2G4w,Waffle House,Urbancrest,OH,43123,"{'OutdoorSeating': 'False', 'RestaurantsAttire...",True,True,False,False,u'none',NaN,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'..."
2,vecuat0jOia-CJveW3ngDw,Schmidt's Sausage Haus,Columbus,OH,43206,"{'GoodForKids': 'True', 'HasTV': 'False', 'Res...",True,True,None,False,u'full_bar',NaN,"{'touristy': None, 'hipster': False, 'romantic...","{'dessert': None, 'latenight': False, 'lunch':..."
3,ZaMMUcOIngBCgxuKxFzfqg,McDonald's,Columbus,OH,43228,"{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",False,True,False,False,u'none',False,"{'romantic': False, 'intimate': False, 'classy...","{'dessert': False, 'latenight': False, 'lunch'..."
4,FrFDIPBzRbFVfanxG4wcNQ,Exotic Latin Grill,Columbus,OH,43202,"{'Alcohol': 'u'none'', 'NoiseLevel': ''average...",True,True,True,False,u'none',False,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': True, 'lunch':..."


##### 2) Breaks out the Ambience column and turns the dictionary into a dataframe

In [49]:
# Creates a list of dictionaries of just the ambience values
ambience_list_of_dicts = list(merged_small_df['att_Ambience'])

In [50]:
ambience_list_of_dicts[:3]

[{'touristy': False,
  'hipster': False,
  'romantic': False,
  'divey': False,
  'intimate': False,
  'trendy': False,
  'upscale': False,
  'classy': True,
  'casual': False},
 {'touristy': False,
  'hipster': False,
  'romantic': False,
  'divey': False,
  'intimate': False,
  'trendy': False,
  'upscale': False,
  'classy': False,
  'casual': False},
 {'touristy': None,
  'hipster': False,
  'romantic': False,
  'divey': False,
  'intimate': False,
  'trendy': False,
  'upscale': False,
  'classy': False,
  'casual': True}]

In [51]:
# Creates a dataframe out of the list of ambience dictionaries
ambience_df = pd.DataFrame()
for idx in range(len(ambience_list_of_dicts)): 
    add_df = pd.DataFrame([ambience_list_of_dicts[idx]])
    ambience_df = pd.concat([ambience_df, add_df])
#     print(idx)

In [52]:
ambience_df.head()

,0,casual,classy,divey,hipster,intimate,romantic,touristy,trendy,upscale
0,NaN,False,True,False,False,False,False,False,False,False
0,NaN,False,False,False,False,False,False,False,False,False
0,NaN,True,False,False,False,False,False,None,False,False
0,NaN,False,False,False,False,False,False,False,False,False
0,NaN,True,False,True,False,False,False,False,False,False


In [53]:
ambience_df[0].value_counts(dropna=False)

NaN    3342
Name: 0, dtype: int64

In [54]:
# Drops the zero column
ambience_df.drop(columns=0, inplace=True)

In [55]:
# resets the index
ambience_df.reset_index(drop=True, inplace=True)

In [56]:
ambience_df.shape

(3342, 9)

In [57]:
ambience_df.head()

,casual,classy,divey,hipster,intimate,romantic,touristy,trendy,upscale
0,False,True,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,None,False,False
3,False,False,False,False,False,False,False,False,False
4,True,False,True,False,False,False,False,False,False


In [58]:
# renames the columns to have'amb_' as as prefix
ambience_df.columns = [f'amb_{col}' for col in ambience_df.columns]

In [59]:
ambience_df.iloc[3320:3324, :]

,amb_casual,amb_classy,amb_divey,amb_hipster,amb_intimate,amb_romantic,amb_touristy,amb_trendy,amb_upscale
3320,True,False,False,False,False,False,False,None,False
3321,False,False,False,False,False,False,False,False,False
3322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3323,False,False,False,False,False,False,False,False,False


In [60]:
ambience_df.iloc[544:547,:]

,amb_casual,amb_classy,amb_divey,amb_hipster,amb_intimate,amb_romantic,amb_touristy,amb_trendy,amb_upscale
544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
546,True,True,False,False,False,False,False,False,False


In [61]:
ambience_df.query('amb_casual == False & amb_classy == False & amb_divey == False & amb_hipster == False & amb_intimate == False & amb_romantic == False & amb_touristy == False & amb_trendy == False & amb_upscale == False')

,amb_casual,amb_classy,amb_divey,amb_hipster,amb_intimate,amb_romantic,amb_touristy,amb_trendy,amb_upscale
1,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False
13,False,False,False,False,False,False,False,False,False
23,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
3328,False,False,False,False,False,False,False,False,False
3332,False,False,False,False,False,False,False,False,False
3335,False,False,False,False,False,False,False,False,False
3336,False,False,False,False,False,False,False,False,False


In [62]:
3342-1036

2306

In [63]:
for col in ambience_df.columns:
    print(col)
    print(ambience_df[col].value_counts(dropna=False))

amb_casual
True     1324
False    1291
NaN       727
Name: amb_casual, dtype: int64
amb_classy
False    2096
NaN       822
True      424
Name: amb_classy, dtype: int64
amb_divey
False    2374
NaN       863
True      105
Name: amb_divey, dtype: int64
amb_hipster
False    2368
NaN       891
True       83
Name: amb_hipster, dtype: int64
amb_intimate
False    2462
NaN       850
True       30
Name: amb_intimate, dtype: int64
amb_romantic
False    2507
NaN       798
True       37
Name: amb_romantic, dtype: int64
amb_touristy
False    2509
NaN       827
True        6
Name: amb_touristy, dtype: int64
amb_trendy
False    2216
NaN       970
True      156
Name: amb_trendy, dtype: int64
amb_upscale
False    2528
NaN       785
True       29
Name: amb_upscale, dtype: int64


In [64]:
ambience_df.isnull().sum()

amb_casual      727
amb_classy      822
amb_divey       863
amb_hipster     891
amb_intimate    850
amb_romantic    798
amb_touristy    827
amb_trendy      970
amb_upscale     785
dtype: int64

#### B) Cleans the GoodForMeal column by separating null rows and not null rows in GoodForMeal column

##### 1) Separates null rows and not null rows in GoodForMeal column

In [65]:
# Splits out the rows that are null and notnull in the att_GoodForMeal column
notnull_meal_df = merged_small_df[merged_small_df['att_GoodForMeal'].notnull()]
null_meal_df = merged_small_df[merged_small_df['att_GoodForMeal'].isnull()]

In [66]:
null_meal_df.head()

,business_id,name,city,state,postal_code,attributes,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_Alcohol,att_RestaurantsTableService,att_Ambience,att_GoodForMeal
5,P0l1XH6KHqo6rk-R03SDXA,White Castle,Grove City,OH,43123,"{'RestaurantsAttire': 'u'casual'', 'BusinessPa...",True,True,False,False,u'none',NaN,"{'touristy': False, 'hipster': False, 'romanti...",NaN
10,ZnZRIs5GmA9VGa_RwFsHaQ,The Bogey Bar & Grill,Powell,OH,43065,"{'RestaurantsPriceRange2': '2', 'Alcohol': 'u'...",True,True,True,True,u'full_bar',NaN,"{'romantic': False, 'intimate': False, 'touris...",NaN
12,ALP7rrHJxXIHyzMMPGQ1vw,LittleMoon Cafe and Tea,Columbus,OH,43202,"{'RestaurantsTakeOut': 'True', 'WheelchairAcce...",NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
14,ukWju178_R0Htx7LBfJFkA,Starbucks,Columbus,OH,43215,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
16,p915L72HheTfBVYWsSfCDg,BrewDog DogTap,Canal Winchester,OH,43110,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...",True,True,True,True,u'full_bar',NaN,"{'touristy': True, 'hipster': True, 'romantic'...",NaN


In [67]:
# Turns each value in att_GoodForMeal column into a literal dictionary
notnull_meal_df['att_GoodForMeal'] = notnull_meal_df.loc[:,'att_GoodForMeal'].map(lambda x: ast.literal_eval(x))

<ipython-input-67-4430bad515d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notnull_meal_df['att_GoodForMeal'] = notnull_meal_df.loc[:,'att_GoodForMeal'].map(lambda x: ast.literal_eval(x))


In [68]:
null_meal_df.shape

(1207, 14)

In [69]:
notnull_meal_df.shape

(2135, 14)

In [70]:
# Checks to make sure the data type is a dictionary type
type(notnull_meal_df['att_GoodForMeal'][0])

dict

In [71]:
# Concatenates the notnull and null att_GoodForMeal dataframes back together
meal_small_df = pd.concat([notnull_meal_df, null_meal_df])

In [72]:
meal_small_df.shape

(3342, 14)

In [73]:
meal_small_df.head()

,business_id,name,city,state,postal_code,attributes,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_Alcohol,att_RestaurantsTableService,att_Ambience,att_GoodForMeal
0,5eq56X-e9YbAKmHgUOyXSg,Hunan Lion,Columbus,OH,43220,"{'GoodForKids': 'True', 'RestaurantsGoodForGro...",True,True,False,True,u'full_bar',True,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'..."
1,Fl_7YDOMt58rTNHa7p2G4w,Waffle House,Urbancrest,OH,43123,"{'OutdoorSeating': 'False', 'RestaurantsAttire...",True,True,False,False,u'none',NaN,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'..."
2,vecuat0jOia-CJveW3ngDw,Schmidt's Sausage Haus,Columbus,OH,43206,"{'GoodForKids': 'True', 'HasTV': 'False', 'Res...",True,True,None,False,u'full_bar',NaN,"{'touristy': None, 'hipster': False, 'romantic...","{'dessert': None, 'latenight': False, 'lunch':..."
3,ZaMMUcOIngBCgxuKxFzfqg,McDonald's,Columbus,OH,43228,"{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",False,True,False,False,u'none',False,"{'romantic': False, 'intimate': False, 'classy...","{'dessert': False, 'latenight': False, 'lunch'..."
4,FrFDIPBzRbFVfanxG4wcNQ,Exotic Latin Grill,Columbus,OH,43202,"{'Alcohol': 'u'none'', 'NoiseLevel': ''average...",True,True,True,False,u'none',False,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': True, 'lunch':..."


In [74]:
# meal_small_df[meal_small_df['att_GoodForMeal'].isnull()]

In [75]:
# Sorts the dataframe by index
meal_small_df.sort_index(inplace=True)

In [76]:
# meal_small_df

##### 2) Breaks out the GoodForMeal column and turns the dictionary into a dataframe

In [77]:
meal_list_of_dicts = list(meal_small_df['att_GoodForMeal'])

In [78]:
len(meal_list_of_dicts)

3342

In [79]:
# meal_list_of_dicts

In [80]:
meal_df = pd.DataFrame()
for idx in range(len(meal_list_of_dicts)): 
    add_meal_df = pd.DataFrame([meal_list_of_dicts[idx]])
    meal_df = pd.concat([meal_df, add_meal_df])
#     print(idx)

In [81]:
meal_df.shape

(3342, 7)

In [82]:
meal_df.head()

,0,breakfast,brunch,dessert,dinner,latenight,lunch
0,NaN,False,False,False,True,False,True
0,NaN,True,False,False,False,False,True
0,NaN,False,None,None,True,False,True
0,NaN,False,False,False,True,False,True
0,NaN,False,False,False,True,True,True


In [83]:
meal_df[0].value_counts(dropna=False)

NaN    3342
Name: 0, dtype: int64

In [84]:
# Drops the zero column
meal_df.drop(columns=0, inplace=True)

In [85]:
# resets the index
meal_df.reset_index(drop=True, inplace=True)

In [86]:
meal_df.shape

(3342, 6)

In [87]:
meal_df.columns = [f'meal_{col}' for col in meal_df.columns]

In [88]:
meal_df.head()

,meal_breakfast,meal_brunch,meal_dessert,meal_dinner,meal_latenight,meal_lunch
0,False,False,False,True,False,True
1,True,False,False,False,False,True
2,False,None,None,True,False,True
3,False,False,False,True,False,True
4,False,False,False,True,True,True


In [89]:
meal_df.query('meal_breakfast == False & meal_brunch == False & meal_dessert == False & meal_dinner == False & meal_latenight == False & meal_lunch == False')

,meal_breakfast,meal_brunch,meal_dessert,meal_dinner,meal_latenight,meal_lunch
6,False,False,False,False,False,False
9,False,False,False,False,False,False
13,False,False,False,False,False,False
22,False,False,False,False,False,False
25,False,False,False,False,False,False
...,...,...,...,...,...,...
3325,False,False,False,False,False,False
3327,False,False,False,False,False,False
3328,False,False,False,False,False,False
3335,False,False,False,False,False,False


In [90]:
3342-683

2659

In [91]:
for col in meal_df.columns:
    print(col)
    print(meal_df[col].value_counts(dropna=False))

meal_breakfast
False    1739
NaN      1396
True      207
Name: meal_breakfast, dtype: int64
meal_brunch
False    1649
NaN      1480
True      213
Name: meal_brunch, dtype: int64
meal_dessert
False    1690
NaN      1528
True      124
Name: meal_dessert, dtype: int64
meal_dinner
NaN      1331
False    1022
True      989
Name: meal_dinner, dtype: int64
meal_latenight
False    1723
NaN      1513
True      106
Name: meal_latenight, dtype: int64
meal_lunch
NaN      1416
True     1032
False     894
Name: meal_lunch, dtype: int64


In [92]:
meal_small_df.head()

,business_id,name,city,state,postal_code,attributes,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_Alcohol,att_RestaurantsTableService,att_Ambience,att_GoodForMeal
0,5eq56X-e9YbAKmHgUOyXSg,Hunan Lion,Columbus,OH,43220,"{'GoodForKids': 'True', 'RestaurantsGoodForGro...",True,True,False,True,u'full_bar',True,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'..."
1,Fl_7YDOMt58rTNHa7p2G4w,Waffle House,Urbancrest,OH,43123,"{'OutdoorSeating': 'False', 'RestaurantsAttire...",True,True,False,False,u'none',NaN,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'..."
2,vecuat0jOia-CJveW3ngDw,Schmidt's Sausage Haus,Columbus,OH,43206,"{'GoodForKids': 'True', 'HasTV': 'False', 'Res...",True,True,None,False,u'full_bar',NaN,"{'touristy': None, 'hipster': False, 'romantic...","{'dessert': None, 'latenight': False, 'lunch':..."
3,ZaMMUcOIngBCgxuKxFzfqg,McDonald's,Columbus,OH,43228,"{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",False,True,False,False,u'none',False,"{'romantic': False, 'intimate': False, 'classy...","{'dessert': False, 'latenight': False, 'lunch'..."
4,FrFDIPBzRbFVfanxG4wcNQ,Exotic Latin Grill,Columbus,OH,43202,"{'Alcohol': 'u'none'', 'NoiseLevel': ''average...",True,True,True,False,u'none',False,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': True, 'lunch':..."


### Combining the ambience df and the GoodForMeal df to the base dataframe that includes the business names

In [93]:
combined_df = pd.concat([meal_small_df, meal_df, ambience_df], axis=1)

In [94]:
with pd.option_context("display.max_columns", 100):
    display(combined_df[:5])

,business_id,name,city,state,postal_code,attributes,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_Alcohol,att_RestaurantsTableService,att_Ambience,att_GoodForMeal,meal_breakfast,meal_brunch,meal_dessert,meal_dinner,meal_latenight,meal_lunch,amb_casual,amb_classy,amb_divey,amb_hipster,amb_intimate,amb_romantic,amb_touristy,amb_trendy,amb_upscale
0,5eq56X-e9YbAKmHgUOyXSg,Hunan Lion,Columbus,OH,43220,"{'GoodForKids': 'True', 'RestaurantsGoodForGro...",True,True,False,True,u'full_bar',True,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",False,False,False,True,False,True,False,True,False,False,False,False,False,False,False
1,Fl_7YDOMt58rTNHa7p2G4w,Waffle House,Urbancrest,OH,43123,"{'OutdoorSeating': 'False', 'RestaurantsAttire...",True,True,False,False,u'none',NaN,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",True,False,False,False,False,True,False,False,False,False,False,False,False,False,False
2,vecuat0jOia-CJveW3ngDw,Schmidt's Sausage Haus,Columbus,OH,43206,"{'GoodForKids': 'True', 'HasTV': 'False', 'Res...",True,True,None,False,u'full_bar',NaN,"{'touristy': None, 'hipster': False, 'romantic...","{'dessert': None, 'latenight': False, 'lunch':...",False,None,None,True,False,True,True,False,False,False,False,False,None,False,False
3,ZaMMUcOIngBCgxuKxFzfqg,McDonald's,Columbus,OH,43228,"{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",False,True,False,False,u'none',False,"{'romantic': False, 'intimate': False, 'classy...","{'dessert': False, 'latenight': False, 'lunch'...",False,False,False,True,False,True,False,False,False,False,False,False,False,False,False
4,FrFDIPBzRbFVfanxG4wcNQ,Exotic Latin Grill,Columbus,OH,43202,"{'Alcohol': 'u'none'', 'NoiseLevel': ''average...",True,True,True,False,u'none',False,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': True, 'lunch':...",False,False,False,True,True,True,True,False,True,False,False,False,False,False,False


In [95]:
with pd.option_context("display.max_columns", 100):
    display(combined_df.tail())

,business_id,name,city,state,postal_code,attributes,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_Alcohol,att_RestaurantsTableService,att_Ambience,att_GoodForMeal,meal_breakfast,meal_brunch,meal_dessert,meal_dinner,meal_latenight,meal_lunch,amb_casual,amb_classy,amb_divey,amb_hipster,amb_intimate,amb_romantic,amb_touristy,amb_trendy,amb_upscale
3337,8n2Lbxf8ts89_OxvXLURiA,Panera Bread,Westerville,OH,43082,"{'RestaurantsPriceRange2': '1', 'Caters': 'Tru...",True,True,True,False,u'none',NaN,"{'touristy': None, 'hipster': None, 'romantic'...","{'dessert': None, 'latenight': None, 'lunch': ...",True,True,None,True,None,True,True,False,None,None,None,False,None,None,None
3338,3nVq6U9HDBcEigqamkrYaw,Harvest Kitchen and Pizzeria - Bexley,Bexley,OH,43209,"{'RestaurantsTableService': 'True', 'NoiseLeve...",True,True,True,False,NaN,True,"{'touristy': None, 'hipster': None, 'romantic'...","{'dessert': None, 'latenight': None, 'lunch': ...",None,None,None,True,None,True,True,None,False,None,None,None,None,True,None
3339,aEIaN73D1hZ5V4zNn_YJiQ,Das KaffeeHaus von Frau Burkhart,Lithopolis,OH,43136,"{'BusinessParking': '{'garage': False, 'street...",True,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3340,hNpnUFVW4Xm5CoShgpey7A,Bistro 2110 At the Blackwell,Columbus,OH,43210,"{'BusinessParking': '{'garage': False, 'street...",True,True,False,True,u'full_bar',NaN,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': None, 'latenight': False, 'lunch':...",True,None,None,True,False,True,False,False,False,False,False,False,False,False,False
3341,cjyyjjBQXp2HdvlHKT_bCw,Scramblers,Columbus,OH,43230,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...",True,True,False,False,u'none',True,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",True,True,False,False,False,True,True,False,False,False,None,False,False,False,False


### Cleaning the combined_df to turn boolean variables into binary vars

In [96]:
# Views the att_alcohol column
combined_df['att_Alcohol'].value_counts(dropna=False)

u'none'             1003
u'full_bar'          796
NaN                  717
'none'               397
'full_bar'           185
u'beer_and_wine'     169
'beer_and_wine'       75
Name: att_Alcohol, dtype: int64

In [97]:
# Creates the att_alcohol dict to change it into a binary variable
has_alcohol_dict = {"u'none'"    : 0, 
                    "u'full_bar'": 1,
                    "'none'"     : 0, 
                    "'full_bar'" : 1,
                    "u'beer_and_wine'": 1,
                    "'beer_and_wine'": 1}

In [98]:
# Reassigns the values in att_alcohol column to be binary
combined_df['att_Alcohol'] = combined_df['att_Alcohol'].map(has_alcohol_dict)

In [99]:
# renames the alcohol column
combined_df.rename(columns={'att_Alcohol': 'att_HasAlcohol'}, inplace=True)

In [100]:
# Creates a list of columns to change into binary 1s and 0s
str_int_cols = ['att_GoodForKids', 'att_RestaurantsGoodForGroups', 'att_OutdoorSeating', 'att_RestaurantsReservations', 'att_RestaurantsTableService']
bool_int_cols = ['meal_breakfast', 'meal_brunch', 'meal_dessert', 'meal_dinner', 'meal_latenight', 'meal_lunch', 'amb_casual', 'amb_classy', 'amb_divey',
                 'amb_hipster', 'amb_intimate', 'amb_romantic', 'amb_touristy', 'amb_trendy', 'amb_upscale']

In [101]:
# Sets the option to see all the columns
pd.set_option('display.max_columns', 30)

In [102]:
# Defines dictionary to convert boolean strings into actual boolean types and boolean types into integers
# string to boolean dictionary
str_bool = {'True': True,
            'False': False}

# boolean to int dictionary
bool_int = {True: int(1),
            False: int(0)}

In [103]:
# combined_df['meal_breakfast'].value_counts()

In [104]:
# combined_df['meal_breakfast'].map(bool_int).value_counts()

In [105]:
# type(combined_df['meal_latenight'][0])

In [106]:
# type(combined_df['amb_upscale'][0])

In [107]:
def convert_TF_to_int(df, cols_str_int, cols_bool_int):
    """Changes the dtype of columns from string --> bool --> int OR
    Changes the dtype of columns from bool --> int"""
    
    # string to boolean dictionary
    str_bool = {'True': True,
                'False': False}

    # boolean to int dictionary
    bool_int = {True: int(1),
                False: int(0)}
    
    for col in cols_str_int: 
        df[col] = df[col].map(str_bool).map(bool_int)
    
    for col in cols_bool_int:
        df[col] = df[col].map(bool_int)
        
    return df

In [108]:
convert_TF_to_int(combined_df, str_int_cols, bool_int_cols).head()

,business_id,name,city,state,postal_code,attributes,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_HasAlcohol,att_RestaurantsTableService,att_Ambience,att_GoodForMeal,meal_breakfast,meal_brunch,meal_dessert,meal_dinner,meal_latenight,meal_lunch,amb_casual,amb_classy,amb_divey,amb_hipster,amb_intimate,amb_romantic,amb_touristy,amb_trendy,amb_upscale
0,5eq56X-e9YbAKmHgUOyXSg,Hunan Lion,Columbus,OH,43220,"{'GoodForKids': 'True', 'RestaurantsGoodForGro...",1.0,1.0,0.0,1.0,1.0,1.0,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Fl_7YDOMt58rTNHa7p2G4w,Waffle House,Urbancrest,OH,43123,"{'OutdoorSeating': 'False', 'RestaurantsAttire...",1.0,1.0,0.0,0.0,0.0,NaN,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,vecuat0jOia-CJveW3ngDw,Schmidt's Sausage Haus,Columbus,OH,43206,"{'GoodForKids': 'True', 'HasTV': 'False', 'Res...",1.0,1.0,NaN,0.0,1.0,NaN,"{'touristy': None, 'hipster': False, 'romantic...","{'dessert': None, 'latenight': False, 'lunch':...",0.0,NaN,NaN,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0
3,ZaMMUcOIngBCgxuKxFzfqg,McDonald's,Columbus,OH,43228,"{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",0.0,1.0,0.0,0.0,0.0,0.0,"{'romantic': False, 'intimate': False, 'classy...","{'dessert': False, 'latenight': False, 'lunch'...",0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FrFDIPBzRbFVfanxG4wcNQ,Exotic Latin Grill,Columbus,OH,43202,"{'Alcohol': 'u'none'', 'NoiseLevel': ''average...",1.0,1.0,1.0,0.0,0.0,0.0,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': True, 'lunch':...",0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
# combined_df.query('amb_casual == 0 & amb_classy == 0 & amb_divey == 0 & amb_hipster == 0 & amb_intimate == 0 & amb_romantic == 0 & amb_touristy == 0 & amb_trendy == 0 & amb_upscale == 0')
combined_df[(combined_df['amb_casual'].isnull()) & (combined_df['amb_classy'].isnull()) & (combined_df['amb_divey'].isnull()) & (combined_df['amb_hipster'].isnull()) & (combined_df['amb_intimate'].isnull()) & (combined_df['amb_romantic'].isnull()) & (combined_df['amb_touristy'].isnull()) & (combined_df['amb_trendy'].isnull()) & (combined_df['amb_upscale'].isnull())]

,business_id,name,city,state,postal_code,attributes,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_HasAlcohol,att_RestaurantsTableService,att_Ambience,att_GoodForMeal,meal_breakfast,meal_brunch,meal_dessert,meal_dinner,meal_latenight,meal_lunch,amb_casual,amb_classy,amb_divey,amb_hipster,amb_intimate,amb_romantic,amb_touristy,amb_trendy,amb_upscale
12,ALP7rrHJxXIHyzMMPGQ1vw,LittleMoon Cafe and Tea,Columbus,OH,43202,"{'RestaurantsTakeOut': 'True', 'WheelchairAcce...",NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,ukWju178_R0Htx7LBfJFkA,Starbucks,Columbus,OH,43215,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,-kcM672owPBvJzDqQ4My_Q,What the Waffle,Columbus,OH,43205,"{'RestaurantsTableService': 'False', 'Restaura...",NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,DwkhuFNZXsnEkox39w99Ug,Sloopy's Pub,Hilliard,OH,43026,"{'OutdoorSeating': 'False', 'HappyHour': 'True...",NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,4KZm9EcnygkFmEWpPSNqIw,Hyatt Regency Columbus,Columbus,OH,43215,"{'RestaurantsReservations': 'True', 'Restauran...",NaN,1.0,NaN,1.0,NaN,NaN,NaN,"{'dessert': False, 'latenight': False, 'lunch'...",0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,gsVv6sRLTC2FWMpQt38EJA,Royal Burger,Columbus,OH,43228,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3319,NcNBXf06ueP47KSaA4Jbig,CMYE Meet The Owner: Beer Tasting & Board Games,Columbus,OH,43212,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3322,sdWYzrG80ZdG010NFEd_gA,Tim Hortons,Columbus,OH,43222,"{'BusinessParking': '{'garage': False, 'street...",1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3333,WxpzviM3pY8CQABSJy0AgA,Starbucks,Upper Arlington,OH,43221,"{'WiFi': 'u'free'', 'BusinessAcceptsCreditCard...",NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
# combined_df.query('meal_breakfast == 0 & meal_brunch == 0 & meal_dessert == 0 & meal_dinner == 0 & meal_latenight == 0 & meal_lunch == 0')
# combined_df[(combined_df['meal_breakfast'].isnull()) & (combined_df['meal_brunch'].isnull()) & (combined_df['meal_dessert'].isnull()) & (combined_df['meal_dinner'].isnull()) & (combined_df['meal_latenight'].isnull()) & (combined_df['meal_lunch'].isnull())]

In [111]:
# prints out the count of each ambience type
meal_cols = ['meal_breakfast', 'meal_brunch', 'meal_dessert', 'meal_dinner', 'meal_latenight', 'meal_lunch']
amb_cols = ['amb_casual', 'amb_classy', 'amb_divey', 'amb_hipster', 'amb_intimate', 'amb_romantic', 'amb_touristy', 'amb_trendy', 'amb_upscale']

for col in amb_cols: 
    length = len(combined_df[(combined_df[col] == 1)])
    print(col)
    print(length)

amb_casual
1324
amb_classy
424
amb_divey
105
amb_hipster
83
amb_intimate
30
amb_romantic
37
amb_touristy
6
amb_trendy
156
amb_upscale
29


In [112]:
# prints out the count of all meal types
for col in meal_cols: 
    length = len(combined_df[(combined_df[col] == 1)])
    print(col)
    print(length)

meal_breakfast
207
meal_brunch
213
meal_dessert
124
meal_dinner
989
meal_latenight
106
meal_lunch
1032


In [113]:
# creates a single unique identifier column
# combined_df.insert(0, 'identifier', combined_df['business_id'] + '|' + combined_df['name'])

In [114]:
# creates the final dataframe for the recommender system by dropping unneeded columns
rec_simple_df = combined_df.drop(columns=['business_id', 'name', 'city', 'state', 'postal_code', 'attributes', 'att_Ambience', 'att_GoodForMeal', 
                                          'meal_breakfast', 'meal_brunch', 'meal_dessert', 'meal_latenight', 'amb_divey', 'amb_hipster', 'amb_intimate',
                                          'amb_romantic', 'amb_touristy', 'amb_trendy', 'amb_upscale'])

In [115]:
# Checks how many are neither casual or classy

# rec_simple_df[(rec_simple_df['amb_casual'] == 0) & (rec_simple_df['amb_classy'] == 0)]

In [116]:
# Checks how many are both casual and classy

# rec_simple_df[(rec_simple_df['amb_casual'] == 1) & (rec_simple_df['amb_classy'] == 1)]

In [117]:
# Checks how many restaurants are either both classy and casual or neither casual nor classy

# rec_simple_df[((rec_simple_df['amb_casual'] == 1) & (rec_simple_df['amb_classy'] == 1)) | ((rec_simple_df['amb_casual'] == 0) & (rec_simple_df['amb_classy'] == 0))]

In [118]:
# Creates a new target variable that maps (1) neither casual nor classy, (2) casual only, (3) classy only, (4) both casual and classy
rec_simple_df['casual_classy'] = rec_simple_df['amb_casual'].astype(str) + ' ' + rec_simple_df['amb_classy'].astype(str)
mapping_amb_dict = {'0.0 0.0':0, '1.0 0.0':1, '0.0 1.0':2, '1.0 1.0':3}
rec_simple_df['amb_target'] = rec_simple_df['casual_classy'].map(mapping_amb_dict)
rec_simple_df.drop(columns='casual_classy', inplace=True)

In [119]:
rec_simple_df.head()

,identifier,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_HasAlcohol,att_RestaurantsTableService,meal_dinner,meal_lunch,amb_casual,amb_classy,amb_target
0,5eq56X-e9YbAKmHgUOyXSg|Hunan Lion,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0
1,Fl_7YDOMt58rTNHa7p2G4w|Waffle House,1.0,1.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0
2,vecuat0jOia-CJveW3ngDw|Schmidt's Sausage Haus,1.0,1.0,NaN,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
3,ZaMMUcOIngBCgxuKxFzfqg|McDonald's,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,FrFDIPBzRbFVfanxG4wcNQ|Exotic Latin Grill,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0


In [120]:
# drops the amb_casual and amb_classy variables after new target is created
rec_simple_df.drop(columns=['amb_casual', 'amb_classy'], inplace=True)

In [128]:
# sets the index to the 'identifier' column
rec_simple_df.set_index('identifier', inplace=True)

In [129]:
rec_simple_df.head()

,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_HasAlcohol,att_RestaurantsTableService,meal_dinner,meal_lunch,amb_target
identifier,,,,,,,,,
5eq56X-e9YbAKmHgUOyXSg|Hunan Lion,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0
Fl_7YDOMt58rTNHa7p2G4w|Waffle House,1.0,1.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0
vecuat0jOia-CJveW3ngDw|Schmidt's Sausage Haus,1.0,1.0,NaN,0.0,1.0,NaN,1.0,1.0,1.0
ZaMMUcOIngBCgxuKxFzfqg|McDonald's,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
FrFDIPBzRbFVfanxG4wcNQ|Exotic Latin Grill,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0


In [130]:
# drops rows that are all null
rec_simple_df.dropna(how='all', inplace=True)

In [131]:
rec_simple_df.shape

(3331, 9)

In [135]:
rec_simple_df[(rec_simple_df['meal_dinner'].isna()) & (rec_simple_df['meal_lunch'].isna())]
# rec_simple_df[(rec_simple_df['meal_dinner'] == 0) & (rec_simple_df['meal_lunch'] == 0)]

,att_GoodForKids,att_RestaurantsGoodForGroups,att_OutdoorSeating,att_RestaurantsReservations,att_HasAlcohol,att_RestaurantsTableService,meal_dinner,meal_lunch,amb_target
identifier,,,,,,,,,
P0l1XH6KHqo6rk-R03SDXA|White Castle,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,2.0
ZnZRIs5GmA9VGa_RwFsHaQ|The Bogey Bar & Grill,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0
ALP7rrHJxXIHyzMMPGQ1vw|LittleMoon Cafe and Tea,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
p915L72HheTfBVYWsSfCDg|BrewDog DogTap,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0
-kcM672owPBvJzDqQ4My_Q|What the Waffle,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
sdWYzrG80ZdG010NFEd_gA|Tim Hortons,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
uRue9xQUNBtu7Ny27j3qyw|Cinnabon,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
zjqOlv9NpVv2509nCYxHWg|Donatos Pizza,1.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,0.0


In [133]:
rec_simple_df.isnull().sum()

att_GoodForKids                  718
att_RestaurantsGoodForGroups     632
att_OutdoorSeating               556
att_RestaurantsReservations      515
att_HasAlcohol                   706
att_RestaurantsTableService     2117
meal_dinner                     1320
meal_lunch                      1405
amb_target                       866
dtype: int64

In [ ]:
# impute NaNs for dinner and lunch --> just assume they serve every meal
# Maybe or maybe not, since can't impute with all 1s because some places
# really are just breakfast

In [143]:
# Exports the dataframe as a csv
rec_simple_df.to_csv('../Data/recommender_table.csv', index_label='identifier')

In [137]:
# business_id to name df
bid_to_name = combined_df.loc[:, ['business_id', 'name']].copy()
bid_to_name.head()

,business_id,name
0,5eq56X-e9YbAKmHgUOyXSg,Hunan Lion
1,Fl_7YDOMt58rTNHa7p2G4w,Waffle House
2,vecuat0jOia-CJveW3ngDw,Schmidt's Sausage Haus
3,ZaMMUcOIngBCgxuKxFzfqg,McDonald's
4,FrFDIPBzRbFVfanxG4wcNQ,Exotic Latin Grill


In [140]:
# Exports the business_id to name dataframe as a csv
bid_to_name.to_csv('../Data/busid_to_name.csv', index=False)